In [1]:
from collections import defaultdict
from os import listdir
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
import re
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from nltk import ngrams
from numpy import log

### Use SpaCy tokenizer API to tokenize the text from the law corpus.

In [3]:
nlp = Polish()
tokenizer = nlp.tokenizer

In [4]:
def get_freq_defaultdict(tokens):
    freq = defaultdict(int)
    for token in tokens:
        if token.text.isalpha():
            freq[token.text] += 1
    return freq

In [5]:
def get_freq_bigrams(tokens):
    freq = defaultdict(int)
    for token in tokens:
        if token[0].text.isalpha() and token[1].text.isalpha():
            freq[token[0].text+' '+token[1].text] += 1
    return freq

In [6]:
global_freq = defaultdict(int) #Aggregate the result to obtain one global frequency list.
global_bigrams = defaultdict(int)
tokens_global = []

for file in listdir("ustawy"):
    with open(f'ustawy/{file}', "r", encoding='utf-8') as f:
        content = f.read().lower()
        content = re.sub(r"\s+", " ", content)
        tokens = tokenizer(content)
        doc_freq = get_freq_defaultdict(tokens)
        [global_freq.update({word: global_freq[word]+doc_freq[word]}) for word in doc_freq]
        bigrams = [gram for gram in ngrams(tokens, 2)]
        doc_freq = get_freq_bigrams(bigrams)
        [global_bigrams.update({word: global_bigrams[word]+doc_freq[word]}) for word in doc_freq]
    

### Use pointwise mutual information to compute the measure for all pairs of words.

In [7]:
bigram_list = []
pmi_list = []
for bigram in global_bigrams:
    bigram_list.append(bigram)
    word_a, word_b = bigram.split()
    pmi_list.append(log((global_bigrams[bigram])/len(global_freq)) - log((global_freq[word_a])/len(global_freq)) - log((global_freq[word_b])/len(global_freq)))
    

### Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [8]:
df =pd.DataFrame({"bigram":bigram_list,"pmi":pmi_list})
df.sort_values(by="pmi",ascending=False)[:10].reset_index(drop=True)

,bigram,pmi
0,pobierały zasiłek,7.747165
1,taksówkami osobowymi,7.747165
2,wagonach kolejowych,7.747165
3,rolniczej ustalonych,7.747165
4,zaopatrzenia emerytalnego,7.747165
5,dotyczących zaopatrzenia,7.747165
6,jednolity tekst,7.747165
7,opieki społecznej,7.747165
8,jakiej prowadzona,7.747165
9,unieważnienie numeru,7.747165


### Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).

In [9]:
global_bigrams_filter = defaultdict(int)
for bigram in global_bigrams:
    if global_bigrams[bigram]>4:
        global_bigrams_filter.update({bigram: global_bigrams_filter[bigram]+global_bigrams[bigram]}) 
        

In [10]:
bigram_list = []
pmi_list = []
for bigram in global_bigrams_filter:
    bigram_list.append(bigram)
    word_a, word_b = bigram.split()
    pmi_list.append(log((global_bigrams[bigram])/len(global_freq)) - log((global_freq[word_a])/len(global_freq)) - log((global_freq[word_b])/len(global_freq)))
df =pd.DataFrame({"bigram":bigram_list,"pmi":pmi_list})
bigram_morf = df.sort_values(by="pmi",ascending=False)[:10].reset_index(drop=True) 
bigram_morf

,bigram,pmi
0,nim następującym,6.137727
1,danej miejscowości,5.955405
2,ubezpieczenia społecznego,5.955405
3,kwartale kalendarzowym,5.801255
4,urząd skarbowy,5.801255
5,górną granicę,5.801255
6,rzeczypospolitej polskiej,5.773084
7,granicę pierwszego,5.618933
8,pierwszego przedziału,5.618933
9,odrębnymi przepisami,5.549940


In [11]:
bigram_list = []
pmi_list = []
for bigram in global_bigrams_filter:
    bigram_list.append(bigram)
    word_a, word_b = bigram.split()
    pmi_list.append(log((global_bigrams[bigram])/len(global_bigrams)) - log((global_freq[word_a])/len(global_bigrams)) - log((global_freq[word_b])/len(global_freq)))
df =pd.DataFrame({"bigram":bigram_list,"pmi":pmi_list})
df.sort_values(by="pmi",ascending=False)[:10].reset_index(drop=True)

,bigram,pmi
0,nim następującym,6.137727
1,ubezpieczenia społecznego,5.955405
2,danej miejscowości,5.955405
3,górną granicę,5.801255
4,kwartale kalendarzowym,5.801255
5,urząd skarbowy,5.801255
6,rzeczypospolitej polskiej,5.773084
7,pierwszego przedziału,5.618933
8,granicę pierwszego,5.618933
9,odrębnymi przepisami,5.549940


### Use Clarin-PL API(https://ws.clarin-pl.eu/tager.shtml) to tag and lemmatize the corpus. Using the tagged corpus compute bigram statistic for the tokens containing: a. lemmatized, downcased word b. morphosyntactic category of the word (subst, fin, adj, etc.)

In [12]:
def get_freq_xml(tokens):
    freq = defaultdict(int)
    for token in tokens:
        if token.split(':')[0].isalpha():
            freq[token] += 1
    return freq
def get_freq_bigrams_xml(tokens):
    freq = defaultdict(int)
    for token in tokens:
        if token[0].split(':')[0].isalpha() and token[1].split(':')[0].isalpha():
            freq[token[0]+' '+token[1]] += 1
    return freq

In [13]:
from xml.dom.minidom import parse

counter_dict = defaultdict(int)
counter_bigram = defaultdict(int)
word_dict = {}
for file in listdir("Wynik"):
    tokens=[]
    with open(f'Wynik/{file}', "r", encoding='utf-8') as f:
        doc = parse(f)
    for token in doc.getElementsByTagName("tok"):
        name = token.getElementsByTagName("orth")[0].firstChild.nodeValue.lower()
        base = token.getElementsByTagName("base")[0].firstChild.nodeValue.lower()
        tag = token.getElementsByTagName("ctag")[0].firstChild.nodeValue.split(':')[0].lower()
        word_dict[name] = {"base": base , "tag": tag}
        tokens.append(base+':'+tag)

    doc_freq = get_freq_xml(tokens)
    [counter_dict.update({word: counter_dict[word]+doc_freq[word]}) for word in doc_freq]
    bigrams = [gram for gram in ngrams(tokens, 2)]
    doc_freq = get_freq_bigrams_xml(bigrams)
    [counter_bigram.update({word: counter_bigram[word]+doc_freq[word]}) for word in doc_freq]


### Compute the same statistics as for the non-lemmatized words (i.e. PMI) and print top-10 entries with at least 5 occurrences.

In [17]:
bigram_list = []
pmi_list = []
for bigram in counter_bigram:
    bigram_list.append(bigram)
    word_a, word_b = bigram.split()
    pmi_list.append(log((counter_bigram[bigram])/len(counter_dict)) - log((counter_dict[word_a])/len(counter_dict)) - log((counter_dict[word_b])/len(counter_dict)))
df =pd.DataFrame({"bigram":bigram_list,"pmi":pmi_list})
bigram_clarin = df.sort_values(by="pmi",ascending=False)[:10].reset_index(drop=True) 
bigram_clarin

,bigram,pmi
0,eegcou:ign eok:ign,10.079036
1,poin:ign formować:inf,10.079036
2,kalen:ign darzowym:ign,10.079036
3,zajmo:ign wanym:ign,10.079036
4,mał:ign żonka:subst,10.079036
5,fomar:ign roulunds:ign,10.079036
6,ty:ppron12 godniu:ign,10.079036
7,uniemoż:ign liwiającej:ign,10.079036
8,krętek:subst blady:adj,10.079036
9,promować:inf nowatorstwo:subst,10.079036


### Compute trigram counts for both corpora and perform the same filtering. Use PMI (with 5 occurrence threshold) to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

##### corp 1

In [18]:
def get_freq_trigrams(tokens):
    freq = defaultdict(int)
    for token in tokens:
        if token[0].text.isalpha() and token[1].text.isalpha() and token[2].text.isalpha():
            freq[token[0].text+' '+token[1].text+' '+token[2].text] += 1
    freq_copy = freq.copy()
    for key in freq:
        if freq_copy[key]<5:
            del freq_copy[key]
            
    return freq_copy

In [19]:
# global_freq = defaultdict(int) #Aggregate the result to obtain one global frequency list.
global_trigrams = defaultdict(int)

for file in listdir("ustawy"):
    with open(f'ustawy/{file}', "r", encoding='utf-8') as f:
        content = f.read().lower()
        content = re.sub(r"\s+", " ", content)
        tokens = tokenizer(content)
        trigrams = [gram for gram in ngrams(tokens, 3)]
        doc_freq = get_freq_trigrams(trigrams)
        [global_trigrams.update({word: global_trigrams[word]+doc_freq[word]}) for word in doc_freq]
    

In [20]:
trigram_list = []
pmi_list = []
for trigram in global_trigrams:
#     print(trigram)
    trigram_list.append(trigram)
    word_a, word_b, word_c = trigram.split(" ")
    pmi_list.append(log((global_trigrams[trigram])/len(global_freq)) - log((global_freq[word_a])/len(global_freq)) - log((global_freq[word_b])/len(global_freq))- log((global_freq[word_c])/len(global_freq)))
 

df =pd.DataFrame({"trigram":trigram_list,"pmi":pmi_list})
trigram_morf = df.sort_values(by="pmi",ascending=False)[:10].reset_index(drop=True)
trigram_morf

,trigram,pmi
0,górną granicę pierwszego,11.574339
1,kwartale kalendarzowym poprzedzającym,11.245835
2,kalendarzowym poprzedzającym termin,11.112303
3,poprzedzającym termin waloryzacji,10.419156
4,po nim następującym,9.807355
5,nim następującym na,9.148694
6,rozumieniu odrębnych przepisów,8.460823
7,następującym na cele,7.867760
8,zwrotu różnicy podatku,7.615796
9,z odrębnymi przepisami,7.535054


##### corp 2

In [21]:
def get_freq_trigrams_xml(tokens):
    freq = defaultdict(int)
    for token in tokens:
        if token[0].split(':')[0].isalpha() and token[1].split(':')[0].isalpha() and token[2].split(':')[0].isalpha():
            freq[token[0]+' '+token[1]+' '+token[2]] += 1
    freq_copy = freq.copy()
    for key in freq:
        if freq_copy[key]<5:
            del freq_copy[key]
            
    return freq_copy

counter_trigram = defaultdict(int)
for file in listdir("Wynik"):
    tokens=[]
    with open(f'Wynik/{file}', "r", encoding='utf-8') as f:
        doc = parse(f)
    for token in doc.getElementsByTagName("tok"):
        name = token.getElementsByTagName("orth")[0].firstChild.nodeValue.lower()
        base = token.getElementsByTagName("base")[0].firstChild.nodeValue.lower()
        tag = token.getElementsByTagName("ctag")[0].firstChild.nodeValue.split(':')[0].lower()
        tokens.append(base+':'+tag)


    trigrams = [gram for gram in ngrams(tokens, 3)]
    doc_freq = get_freq_trigrams_xml(trigrams)
    [counter_trigram.update({word: counter_trigram[word]+doc_freq[word]}) for word in doc_freq]
counter_trigram

defaultdict(int,
            {'o:prep służba:subst wojskowy:adj': 22,
             'służba:subst wojskowy:adj żołnierz:subst': 38,
             'wojskowy:adj żołnierz:subst zawodowy:adj': 31,
             'do:prep zawodowy:adj służba:subst': 41,
             'zawodowy:adj służba:subst wojskowy:adj': 377,
             'służbowy:adj żołnierz:subst zawodowy:adj': 6,
             'uposażenie:subst i:conj inny:subst': 29,
             'i:conj inny:subst należność:subst': 26,
             'inny:subst należność:subst pieniężny:adj': 23,
             'przez:prep żołnierz:subst zawodowy:adj': 26,
             'żołnierz:subst zawodowy:adj z:prep': 26,
             'zawodowy:adj z:prep zawodowy:adj': 6,
             'z:prep zawodowy:adj służba:subst': 139,
             'kandydat:subst na:prep żołnierz:subst': 20,
             'na:prep żołnierz:subst zawodowy:adj': 20,
             'żołnierz:subst zawodowy:adj i:conj': 34,
             'żołnierz:subst zawodowy:adj w:prep': 23,
             'w:prep

In [22]:
trigram_list = []
pmi_list = []
for trigram in counter_trigram:
    trigram_list.append(trigram)
    word_a, word_b, word_c = trigram.split(" ")
    pmi_list.append(log((counter_trigram[trigram])/len(counter_dict)) - log((counter_dict[word_a])/len(counter_dict)) - log((counter_dict[word_b])/len(counter_dict))- log((counter_dict[word_c])/len(counter_dict)))
df =pd.DataFrame({"trigram":trigram_list,"pmi":pmi_list})
trigram_clarina = df.sort_values(by="pmi",ascending=False)[:10].reset_index(drop=True) 
trigram_clarina

,trigram,pmi
0,reakcja:subst łańcuchowa:subst rozszczepienie:...,14.728727
1,kurtka:subst anorak:subst etc:ign,13.671912
2,piłka:subst nożny:adj uefa:subst,13.576047
3,finałowy:adj turniej:subst mistrzostwa:subst,13.380046
4,milion:brev dolar:subst usa:subst,13.046793
5,profil:subst zaufany:adj epuap:ign,13.030078
6,uniwersytet:subst maria:subst curie:subst,12.786583
7,rektyfikować:ppas zagęścić:ppas moszcz:subst,12.705206
8,łańcuchowa:subst rozszczepienie:subst jądrowy:adj,12.579416
9,turniej:subst mistrzostwa:subst europa:subst,12.547137


### Create a table comparing the results for copora without and with tagging and lemmatization (separate table for bigrams and trigrams).

##### Bigrams

In [32]:
pd.DataFrame({"spy_cy":bigram_morf['bigram'].values, "clarin":bigram_clarin['bigram'].values})

,spy_cy,clarin
0,nim następującym,eegcou:ign eok:ign
1,danej miejscowości,poin:ign formować:inf
2,ubezpieczenia społecznego,kalen:ign darzowym:ign
3,kwartale kalendarzowym,zajmo:ign wanym:ign
4,urząd skarbowy,mał:ign żonka:subst
5,górną granicę,fomar:ign roulunds:ign
6,rzeczypospolitej polskiej,ty:ppron12 godniu:ign
7,granicę pierwszego,uniemoż:ign liwiającej:ign
8,pierwszego przedziału,krętek:subst blady:adj
9,odrębnymi przepisami,promować:inf nowatorstwo:subst


#### trigram

In [34]:
pd.DataFrame({"spy_cy":trigram_morf['trigram'].values, "clarin":trigram_clarina['trigram'].values})

,spy_cy,clarin
0,górną granicę pierwszego,reakcja:subst łańcuchowa:subst rozszczepienie:...
1,kwartale kalendarzowym poprzedzającym,kurtka:subst anorak:subst etc:ign
2,kalendarzowym poprzedzającym termin,piłka:subst nożny:adj uefa:subst
3,poprzedzającym termin waloryzacji,finałowy:adj turniej:subst mistrzostwa:subst
4,po nim następującym,milion:brev dolar:subst usa:subst
5,nim następującym na,profil:subst zaufany:adj epuap:ign
6,rozumieniu odrębnych przepisów,uniwersytet:subst maria:subst curie:subst
7,następującym na cele,rektyfikować:ppas zagęścić:ppas moszcz:subst
8,zwrotu różnicy podatku,łańcuchowa:subst rozszczepienie:subst jądrowy:adj
9,z odrębnymi przepisami,turniej:subst mistrzostwa:subst europa:subst


### Answer the following questions:

#### Why do we have to filter the bigrams, rather than the token sequence?
Aby zachować sens zdań. Jeżeli najpierw byśmy filtrowali tokeny, to potem powstałyby nam bigramy ze słów, które logicznie w zdaniu nie powinny być obok siebie np "Ala ma kota. Kot jest gruby". Jeżeli usuniemy kropke najpierw to otrzymamy bigram "kot kot", jeżeli będziemy filtrować bigramy to nie otrzymamy takiego połączenia, czego oczekujemy.

#### Which method works better for the bigrams and which for the trigrams?
Lepiej działa dla trigramów tokenizacja za pomocą clarin, ponieważ uwzględnia również odmiany wyrazów, ponieważ wszystkie człony sprowadzamy do formy podstawowej. Jednak dla bigramów nie zadziałał już najlepiej

#### What types of expressions are discovered by the methods.
Wyrażenie przyimkowe, epitety, nazwy własne np instytucji


#### Can you devise a different type of filtering that would yield better results?
nie